# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Подключаем библиотеки:

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Загрузим данные из файлов

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Изучим содержимое файлов. Для удобства, напишем функцию, которая будет выводить первые десять строк датафрейма, инфо, и количество дубликатов

In [3]:
def data_info(data_name):
    display(data_name.head(10))
    data_name.info()
    display('Количество дубликатов', sum(data_name.duplicated()))
    
data_info(data_0)
data_info(data_1)
data_info(data_2)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


'Количество дубликатов'

0

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


...

0

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


'Количество дубликатов'

0

Все три датафрейма имеют одинаковый размер, в них нет дубликатов и пропусков, и нет необходимости в предобработке. Необходимо только произвести разбивку на выборки. Product- целевой признак, f0, f1 и f2 - признаки, id для нашей задачи не нужен.

In [4]:
feature_names = ['f0', 'f1', 'f2']
features_0 = data_0[feature_names]
target_0 = data_0['product']
features_1 = data_1[feature_names]
target_1 = data_1['product']
features_2 = data_2[feature_names]
target_2 = data_2['product']

Выводы:

1) Импортированы библиотеки, загружены данные.

2) Содержимое датафреймов проанализировано, предобработка данным не требуется, нет пропусков и дубликатов.

3) Выполнена разбивка на признаки и целевой признак.

## Обучение и проверка модели

Разделим данные на обучающую и валидационную выборку с соотношением 75:25. Для разбивки, создадим функцию.

Хранить промежуточные данные (результаты разбивки)- нам необязательно. По условиям задачи, нам надо сохранить предсказания на валидационной выборке и правильные ответы. Включим в функцию обучение модели, результат, который она будет выводить- предсказания (predict) и правильные ответы(target_valid). 

In [5]:
def model_split(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)

    return predict, target_valid

Теперь нужно сохранить для каждого региона результаты в отдельную переменную, и вывести на экран средний запас предсказанного сырья и RMSE модели.

In [6]:
predict_0, target_0_valid = model_split(features_0, target_0)
predict_0_mean = predict_0.mean()
rmse_0 = mean_squared_error(target_0_valid, predict_0)**.5
display('Cредний запас предсказанного сырья в регионе 0', predict_0_mean)
display('RMSE модели', rmse_0)

'Cредний запас предсказанного сырья в регионе 0'

92.59256778438035

'RMSE модели'

37.5794217150813

Сделаем то же самое для региона 1

In [7]:
predict_1, target_1_valid = model_split(features_1, target_1)
predict_1_mean = predict_1.mean()
rmse_1 = mean_squared_error(target_1_valid, predict_1)**.5
display('Cредний запас предсказанного сырья в регионе 1', predict_1_mean)
display('RMSE модели', rmse_1)

'Cредний запас предсказанного сырья в регионе 1'

68.728546895446

'RMSE модели'

0.893099286775617

Сделаем то же самое для региона 2

In [8]:
predict_2, target_2_valid = model_split(features_2, target_2)
predict_2_mean = predict_2.mean()
rmse_2 = mean_squared_error(target_2_valid, predict_2)**.5
display('Cредний запас предсказанного сырья в регионе 2', predict_2_mean)
display('RMSE модели', rmse_2)

'Cредний запас предсказанного сырья в регионе 2'

94.96504596800489

'RMSE модели'

40.02970873393434

Выводы:

1) Провели разбивку на выборки, обучение модели

2) Сохранили результаты в отдельные переменные, вывели на экран средние предсказанные запасы и RMSE.

3) У регионов 0 и 2 высокие значения предсказанных запасов, и высокий RMSE- как следствие, низкая надежность результатов

4) У региона 1 предсказанные запасы ниже, но и RMSE низкий- что говорит о хорошей точности предсказаний

## Подготовка к расчёту прибыли

Посчитаем минимальный объем продукта для безубыточности скважин.

Формула выглядит так:

минимальный объем продукта = бюджет / (количество скважин * цена единицы продукта)

Все ключевые параметры, используемые для расчета прибыли, сохраним в отдельных переменных.

В исходных данных, единица объема- это тысяча баррелей, поэтому и цену сразу укажем за тысячу баррелей

Кроме того, создадим переменную с количеством разведываемых скважин в регионе (500). Она не нужна на данном этапе, но понадобится на следующем.

In [9]:
BUDGET = 10000000000 #бюджет
BEST_WELLS = 200 #количество лучших скважин, отобранных после разведки
PRICE = 450000 #цена тысячи баррелей нефти
NUMBER_OF_WELLS = 500 #количество разведываемых скважин.
min_volume_product = BUDGET / (BEST_WELLS * PRICE)
display('Минимальный объем продукта', min_volume_product)

'Минимальный объем продукта'

111.11111111111111

Выводы: 
Результат превышает средний объем скважин в любом из регионов.

## Расчёт прибыли и рисков 

Для каждого региона, нам нужно случайно отбирать по 500 скважин, и считать средний объем 200 лучших, суммарный объем 200 лучших, и размер прибыли. Напишем функцию, которая будет это делать:

In [10]:
def predicted_income(target, predicts):

    sample_predictions = predicts.sample(n = NUMBER_OF_WELLS, random_state=12345)
    top_predictions = sample_predictions.sort_values(ascending=False)[:BEST_WELLS]
    top_targets = target[top_predictions.index]
    top_wells_mean = top_targets.mean()
    volume = sum(top_targets)
    income = volume * PRICE - BUDGET
    display('Средний запас сырья среди скважин с максимальным показателем:', top_wells_mean)
    display('Суммарный целевой объём сырья:', volume)
    display('Прибыль для полученного объёма сырья:', income)

Подготовим исходные данные, чтобы функция могла с ними работать: удалим индексы в target_valid, применим reshape к predict, чтобы изменить форму массива.

In [11]:
predict_0 = pd.Series(*predict_0.reshape(1,-1))
predict_1 = pd.Series(*predict_1.reshape(1,-1))
predict_2 = pd.Series(*predict_2.reshape(1,-1))
target_0_valid = pd.Series(target_0_valid).reset_index(drop=True)
target_1_valid = pd.Series(target_1_valid).reset_index(drop=True)
target_2_valid = pd.Series(target_2_valid).reset_index(drop=True)

Теперь посчитаем результаты для нулевого региона:

In [12]:
predicted_income(target_0_valid, predict_0)

'Средний запас сырья среди скважин с максимальным показателем:'

118.65632064324916

'Суммарный целевой объём сырья:'

23731.26412864983

'Прибыль для полученного объёма сырья:'

679068857.8924236

Для первого региона:

In [13]:
predicted_income(target_1_valid, predict_1)

'Средний запас сырья среди скважин с максимальным показателем:'

119.77199871307573

'Суммарный целевой объём сырья:'

23954.3997426152

'Прибыль для полученного объёма сырья:'

779479884.1768398

Для второго региона:

In [14]:
predicted_income(target_2_valid, predict_2)

'Средний запас сырья среди скважин с максимальным показателем:'

115.99989047802423

'Суммарный целевой объём сырья:'

23199.978095604856

'Прибыль для полученного объёма сырья:'

439990143.0221844

Теперь используем Bootstrap, и найдем распределение прибыли на 1000 выборок. 

Согласно условиям, нам надо найти 95-% доверительный интервал. Риск убытков в искомом регионе должен не превышать 2,5%

Нам понадобится модифицированная функция predicted_income: она должна не выводить результат на экран, а возвращать переменную income.

In [15]:
def predicted_income_m(target, predicts, state):

    sample_predictions = predicts.sample(n = NUMBER_OF_WELLS, replace = True, random_state = state)
    top_predictions = sample_predictions.sort_values(ascending=False)[:BEST_WELLS]
    top_targets = target[top_predictions.index]
    volume = sum(top_targets)
    income = volume * PRICE - BUDGET
    return income

Теперь создадим еще одну функцию risks_calc- для расчета рисков. В ней будет цикл, который сделает нам 1000 выборок, для каждой из них будет вызываться функция predicted_income_m, результаты которой будут записываться в переменную incomes. Далее, функция risks_calc расчитает доверительный интервал и риск убытков, на основе полученных данных.

In [16]:
def risks_calc(target, predicts):
    bootstrap_s = 1000 #количество выборок, получаемых по технике bootstrap
    alpha = 0.05 
    incomes = []
    state = np.random.RandomState(12345)
    for n in range(bootstrap_s):
        income = predicted_income_m(target, predicts, state)
        incomes.append(income)
    incomes = pd.Series(incomes)
    income_mean = incomes.mean()
    conf_int_left = incomes.quantile(alpha/2)
    conf_int_right = incomes.quantile(1 - alpha/2)
    
    #loss_count = 0
    #for inc in incomes :
    #    if inc < 0 :
    #        loss_count += 1
    risk_rate = (incomes < 0).mean()
        
    display('Средняя прибыль {:.2f} (млн. Р)'.format(income_mean / 10**6))
    display('95% доверительный интервал: {:.2f} : {:.2f} (млн. Р)'.format(conf_int_left / 10**6, conf_int_right / 10**6))
    display('Риск убытков: {:.2f} %'.format(risk_rate * 100))

Посчитаем с помощью этой функции риски для каждого из трех регионов:

In [17]:
risks_calc(target_0_valid, predict_0)

'Средняя прибыль 396.16 (млн. Р)'

'95% доверительный интервал: -111.22 : 909.77 (млн. Р)'

'Риск убытков: 6.90 %'

In [18]:
risks_calc(target_1_valid, predict_1)

'Средняя прибыль 456.05 (млн. Р)'

'95% доверительный интервал: 33.82 : 852.29 (млн. Р)'

'Риск убытков: 1.50 %'

In [19]:
risks_calc(target_2_valid, predict_2)

'Средняя прибыль 404.40 (млн. Р)'

'95% доверительный интервал: -163.35 : 950.36 (млн. Р)'

'Риск убытков: 7.60 %'

Выводы:

1) Создали функцию, с помощью посчитали прибыльность 200 лучших скважин из случайных 500.

2) При выборке случайных 500 скважин, удалось найти 200 со средним содержанием объема сырья выше целевого, во всех трех регионах.

3) Как результат, во всех трех выборках, была прибыль, а не убыток.

4) После этого, с помощью техники Bootstrap, посчитали риск убытков во всех трех регионах.

5) В регионах 0 и 2, риск убытков выше 2,5%

6) В регионе 1, риск убытков 1,5%

7) Как результат, для разработки реомендуется регион 1.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован